In [ ]:
#Ignore warnings
import warnings
warnings.simplefilter('ignore')

In [ ]:
# Importing the tensorflow,Keras libraries 
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Input    
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace

# Importing opencv libraries 
import imutils
import cv2
from imutils.video import VideoStream

# Importing the basic libraries
import time
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

print("Libraries imported")

In [ ]:
# load our serialized face detector 
print("[INFO] loading face detector...")
protoPath = "face_detection_model/deploy.prototxt"
modelPath = "face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print("Done")
#Create directory to save original image along with recognized faces.
if not os.path.exists("outputs"):
    os.mkdir("outputs")
    

In [ ]:
warnings.simplefilter('ignore')
#for time calculation %time
#load model
from keras.models import load_model
model = load_model('models/faceacc3.hdf5')
# model.summary()

Saves Images to output directory.It will show bounding box in output screen.
Press 'c' to perform the recognition.

In [ ]:
# initialize the video stream, then allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
#time delay for loading the camera
time.sleep(2.0)
#for counting the unknown people
j=0
# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    frame = vs.read()
    # resize the frame to have a width of 800 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    frame = imutils.resize(frame, width=800)
    (h, w) = frame.shape[:2]
    #A variable to hold the press event of key 'c' (initialy False)
    save = False
    #Capturing the key Strokes
    key = cv2.waitKey(1) & 0xFF
    #If c is pressed save the faces    
    if key == ord("c"):
        #create a folder name with time stamp
        original = str(time.strftime("%Y-%m-%d_%H-%M-%S"))
        #creating the directory
        os.mkdir("outputs/"+original)
        #Save the original image        
        cv2.imwrite("outputs/"+str(original)+"/"+"original.jpg",frame) 
        #Variable that helps in saving the face images
        save = True
        
        print("done")


    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # extract the face ROI
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue

            #face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            #Resize the image as per the model dimensions
            test_image = cv2.resize(face,(300,240))
            #Convert image to array
            test_image = image.img_to_array(test_image)
            #Convert image to single dimensional array
            test_image = np.expand_dims(test_image, axis = 0)
            #Predict the image
            result = model.predict(test_image)
            #probability estimation
            proba = model.predict_proba(test_image/255)
            #Class estimation
            predicted_classes = np.argmax(proba,axis=1)
#             print(result)
            if result[0][0]:
                name = "Aka"
            elif result[0][1]:
                name = "Pramod"
            elif result[0][2]:
                name = "Rahul"
            elif result[0][3]:
                name = "Shreyas"
            elif result[0][4]:
                name = "Unknown"
            if name == "Unknown":
                name = "Unknown"+str(i)+" "

            #Save the faces only key 'c' is pressed
            if save:
                cv2.imwrite("outputs/"+str(original)+"/"+str(name)+".jpg",face) 

            # draw the bounding box of the face along with the
            # associated probability
            text = "{}: {:.2f}%".format(name, proba[0][predicted_classes[0]] * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 2)
        
    # show the output frame
    cv2.imshow("Frame", frame)
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()

Saves only Images to output directory.It will not show bounding box in output screen.
Press 'c' to perform the recognition.

In [ ]:
# Initialize the video stream, then allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
#time delay for loading the camera
time.sleep(2.0)
#for counting the unknown people
j=0
# loop over frames from the video file stream
while True:
    # grab the frame from the  video stream
    frame = vs.read()
    # resize the frame to have a width of 800 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    frame = imutils.resize(frame, width=800)
    (h, w) = frame.shape[:2]    
    #Capturing the key Strokes
    key = cv2.waitKey(1) & 0xFF
    #If c is pressed perform detection of faces
    if key == ord("c"):
        #create a folder name with time stamp
        original = str(time.strftime("%Y-%m-%d_%H-%M-%S"))
        #creating the directory
        os.mkdir("outputs/"+original)
        #Save the original image
        cv2.imwrite("outputs/"+str(original)+"/"+"original.jpg",frame) 
        
        print("done")

        # construct a blob from the image
        imageBlob = cv2.dnn.blobFromImage(
            cv2.resize(frame, (300, 300)), 1.0, (300, 300),
            (104.0, 177.0, 123.0), swapRB=False, crop=False)

        # apply OpenCV's deep learning-based face detector to localize
        # faces in the input image
        detector.setInput(imageBlob)
        detections = detector.forward()
        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                # extract the face ROI
                face = frame[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]
                # ensure the face width and height are sufficiently large
                if fW < 20 or fH < 20:
                    continue
 
                #face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
                #Resize the image as per the model dimensions
                test_image = cv2.resize(face,(300,240))
                #Convert image to array
                test_image = image.img_to_array(test_image)
                #Convert image to single dimensional array
                test_image = np.expand_dims(test_image, axis = 0)
                #Predict the image
                result = model.predict(test_image)
                #probability estimation
                proba = model.predict_proba(test_image/255)
                #Class estimation
                predicted_classes = np.argmax(proba,axis=1)
                #print(result)
                if result[0][0]:
                    name = "Akashya"
                elif result[0][1]:
                    name = "Pramod"
                elif result[0][2]:
                    name = "Rahul"
                elif result[0][3]:
                    name = "Shreyas"
                elif result[0][4]:
                    name = "Unknown"
                if name == "Unknown":
                    name = "Unknown"+str(i)+" "
                

                #Save the face images 
                cv2.imwrite("outputs/"+str(original)+"/"+str(name)+".jpg",face) 

        
    # show the output frame
    cv2.imshow("Frame", frame)
    #cv2.imshow("face", face)


    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

#Relase all resourses
cv2.destroyAllWindows()
vs.stop()


In [ ]:
from PIL import ImageTk, Image

In [ ]:
from tkinter import *
from PIL import ImageTk, Image
import cv2
import tkinter
root = Tk()
# Create a frame
app = Frame(root, bg="white")
app.grid()
# Create a label in the frame
lmain = Label(app)
lmain.grid(row=0,column = 0)

# Capture from camera
cap = cv2.VideoCapture(0)

# function for video streaming
def video_stream():
    _, frame = cap.read()
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(1, video_stream) 

    
def helloCallBack():
    tkMessageBox.showinfo( "Hello Python", "Hello World")
    
B = tkinter.Button(root, text ="Capture", command = video_stream)
B.grid(row=1)    


scrollbar = Scrollbar(root,orient=VERTICAL)
scrollbar.pack(side = RIGHT, anchor = NW,fill="x")


# B = tkinter.Button(root, text ="imge", command = video_stream)
# B.grid(row=0,column = 1)    

# video_stream()
root.mainloop()

In [ ]:
from __future__ import print_function
from PIL import Image
from PIL import ImageTk
import tkinter as tki
import threading
import datetime
import imutils
import cv2
import os

class PhotoBoothApp:
    def __init__(self, vs, outputPath=r"C:\Users\dell5567\Desktop\projectF\outputs"):
        # store the video stream object and output path, then initialize
        # the most recently read frame, thread for reading frames, and
        # the thread stop event
        self.vs = vs
        self.outputPath = outputPath
        self.frame = None
        self.thread = None
        self.stopEvent = None
        # initialize the root window and image panel
        self.root = tki.Tk()
        self.panel = None

        # create a button, that when pressed, will take the current
        # frame and save it to file
        btn = tki.Button(self.root, text="Snapshot!",
            command=self.takeSnapshot)
        btn.pack(side="bottom", fill="both", expand="yes", padx=10,
            pady=10)
        
        
#         !!!!!!!!!!
        scrollbar = tki.Scrollbar(self.root)
        scrollbar.pack(side="right", fill="y") 

         
        
#         !!!!!!!!!!!!!!!!!!!




        # start a thread that constantly pools the video sensor for
        # the most recently read frame
        self.stopEvent = threading.Event()
        self.thread = threading.Thread(target=self.videoLoop, args=())
        self.thread.start()
        # set a callback to handle when the window is closed
#         self.root.wm_title("PyImageSearch PhotoBooth")
#         self.root.wm_protocol("WM_DELETE_WINDOW", self.onClose)

    def videoLoop(self):
        # DISCLAIMER:
        # I'm not a GUI developer, nor do I even pretend to be. This
        # try/except statement is a pretty ugly hack to get around
        # a RunTime error that Tkinter throws due to threading
        try:
            # keep looping over frames until we are instructed to stop
            while not self.stopEvent.is_set():
                # grab the frame from the video stream and resize it to
                # have a maximum width of 300 pixels
                self.frame = self.vs.read()
                self.frame = imutils.resize(self.frame, width=400)

                # OpenCV represents images in BGR order; however PIL
                # represents images in RGB order, so we need to swap
                # the channels, then convert to PIL and ImageTk format
                image = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(image)
                image = ImageTk.PhotoImage(image)

                # if the panel is not None, we need to initialize it
                if self.panel is None:
                    self.panel = tki.Label(image=image)
                    self.panel.image = image
                    self.panel.pack(side="left", padx=10, pady=10)

                # otherwise, simply update the panel
                else:
                    self.panel.configure(image=image)
                    self.panel.image = image
        except RuntimeError:            
            print("[INFO] caught a RuntimeError")

    def takeSnapshot(self):
        # grab the current timestamp and use it to construct the
        # output path
        ts = datetime.datetime.now()
        filename = "{}.jpg".format(ts.strftime("%Y-%m-%d_%H-%M-%S"))
        p = os.path.sep.join((self.outputPath, filename))
        # save the file
        cv2.imwrite(p, self.frame.copy())
        print("[INFO] saved {}".format(filename))

#     def onClose(self):
#         # set the stop event, cleanup the camera, and allow the rest of
#         # the quit process to continue
#         print("[INFO] closing...")
#         self.stopEvent.set()
#         self.vs.stop()
#         self.root.quit()
        
from __future__ import print_function
# from pyimagesearch.photoboothapp import PhotoBoothApp
from imutils.video import VideoStream
import argparse
import time

# initialize the video stream and allow the camera sensor to warmup
print("[INFO] warming up camera...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
# start the app
pba = PhotoBoothApp(vs)
pba.root.mainloop()

In [ ]:
from __future__ import print_function
from PIL import Image
from PIL import ImageTk
import tkinter as tki
import threading
import datetime
import imutils
import cv2
import os

class PhotoBoothApp:
    def __init__(self, vs, outputPath=r"C:\Users\dell5567\Desktop\projectF\outputs"):
        # store the video stream object and output path, then initialize
        # the most recently read frame, thread for reading frames, and
        # the thread stop event
        self.vs = vs
        self.outputPath = outputPath
        self.frame = None
        self.thread = None
        self.stopEvent = None
        # initialize the root window and image panel
        self.root = tki.Tk()
        
        ## Grid sizing behavior in window
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)
        self.cnv = tki.Canvas(self.root)
        
        
        
        
        
        self.panel = None

        # create a button, that when pressed, will take the current
        # frame and save it to file
        btn = tki.Button(self.root, text="Snapshot!",
            command=self.takeSnapshot)
        btn.grid(row=1, column=0)
 


         
        
        ## Frame in canvas
        self.frm = tki.Frame(self.cnv).grid(row=0, column=1)
        vScroll = tki.Scrollbar(self.frm, orient="vertical")
        vScroll.grid(row=0,column=2)        
        
        ## This puts the frame in the canvas's scrollable zone
        self.cnv.create_window(0, 0, window=self.frm)
        ## Frame contents
        image = Image.open(r"C:\Users\dell5567\Desktop\arc1.png")
        
        
        tkimage = ImageTk.PhotoImage(image)
        myvar=tki.Label(self.frm,image = tkimage)
        myvar.image = tkimage
        myvar.grid()
        
        
        
#         photo = ImageTk.PhotoImage(image)
#         label = tki.Label(image=photo)
#         label.image = photo # keep a reference!
#         label.grid(row=0, column=1)

#         label.image = photo
#         label.grid(row=1, column=1)



        
        
#         !!!!!!!!!!
        
#         !!!!!!!!!!!!!!!!!!!




        # start a thread that constantly pools the video sensor for
        # the most recently read frame
        self.stopEvent = threading.Event()
        self.thread = threading.Thread(target=self.videoLoop, args=())
        self.thread.start()
        # set a callback to handle when the window is closed
#         self.root.wm_title("PyImageSearch PhotoBooth")
#         self.root.wm_protocol("WM_DELETE_WINDOW", self.onClose)

    def videoLoop(self):
        # DISCLAIMER:
        # I'm not a GUI developer, nor do I even pretend to be. This
        # try/except statement is a pretty ugly hack to get around
        # a RunTime error that Tkinter throws due to threading
        try:
            # keep looping over frames until we are instructed to stop
            while not self.stopEvent.is_set():
                # grab the frame from the video stream and resize it to
                # have a maximum width of 300 pixels
                self.frame = self.vs.read()
                self.frame = imutils.resize(self.frame, width=400)

                # OpenCV represents images in BGR order; however PIL
                # represents images in RGB order, so we need to swap
                # the channels, then convert to PIL and ImageTk format
                image = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(image)
                image = ImageTk.PhotoImage(image)

                # if the panel is not None, we need to initialize it
                if self.panel is None:
                    self.panel = tki.Label(image=image)
                    self.panel.image = image
                    self.panel.grid(row=0, column=0)

                # otherwise, simply update the panel
                else:
                    self.panel.configure(image=image)
                    self.panel.image = image
        except RuntimeError:            
            print("[INFO] caught a RuntimeError")

    def takeSnapshot(self):
        # grab the current timestamp and use it to construct the
        # output path
        ts = datetime.datetime.now()
        filename = "{}.jpg".format(ts.strftime("%Y-%m-%d_%H-%M-%S"))
        p = os.path.sep.join((self.outputPath, filename))
        # save the file
        cv2.imwrite(p, self.frame.copy())
        print("[INFO] saved {}".format(filename))

#     def onClose(self):
#         # set the stop event, cleanup the camera, and allow the rest of
#         # the quit process to continue
#         print("[INFO] closing...")
#         self.stopEvent.set()
#         self.vs.stop()
#         self.root.quit()
        
from __future__ import print_function
# from pyimagesearch.photoboothapp import PhotoBoothApp
from imutils.video import VideoStream
import argparse
import time

# initialize the video stream and allow the camera sensor to warmup
print("[INFO] warming up camera...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
# start the app
pba = PhotoBoothApp(vs)
pba.root.mainloop()

In [ ]:
# import tkinter and all its functions
from tkinter import * 

root = Tk() # create root window
root.title("GUI Layout") # title of the GUI window
root.maxsize(1000, 600) # specify the max size the window can expand to
root.config(bg="skyblue") # specify background color
root.geometry("1000x600")

# root.columnconfigure(0, weight=0)
# root.rowconfigure(0, weight=0)   


# Create left and right frames
left_frame = Frame(root, width=750, height=600, bg='grey')
left_frame.place(x=0, y=0)


# left_frame.columnconfigure(0, weight=0)
# left_frame.rowconfigure(0, weight=0)
        
        
        
right_frame = Frame(root, width=250, height=600, bg='red')
right_frame.place(x=750, y=0)

# Create frames and labels in left_frame
# Label(left_frame, text="Original Image").grid(row=0, column=0, padx=1, pady=1)

# load image to be "edited"
image = PhotoImage(r"C:\Users\dell5567\Desktop\arc1.png")
original_image = image.subsample(3,3) # resize image using subsample

# Display image in left_frame
Label(left_frame, image=image,justify="center").place(x=100,y=0)

# Display image in right_frame
# Label(right_frame, image=original_image).grid(row=0, column=0, padx=1, pady=1)




listbox = Listbox(right_frame,width=30,height=35)
# listbox.place(x=2,y=2)
listbox.pack(side="left",fill="y")


scrollbar = Scrollbar(right_frame,orient="vertical")
scrollbar.pack(fill="y")

for i in range(100):
    listbox.insert(END, i)

# attach listbox to scrollbar
listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=listbox.yview)










root.mainloop()

In [ ]:
image


In [ ]:
# import tkinter and all its functions
from tkinter import * 

root = Tk() # create root window
root.title("Basic GUI Layout") # title of the GUI window
root.maxsize(900, 600) # specify the max size the window can expand to
root.config(bg="skyblue") # specify background color

# Create left and right frames
left_frame = Frame(root, width=200, height=400, bg='grey')
left_frame.grid(row=0, column=0, padx=10, pady=5)
right_frame = Frame(root, width=650, height=400, bg='grey')
right_frame.grid(row=0, column=1, padx=10, pady=5)

# Create frames and labels in left_frame
Label(left_frame, text="Original Image").grid(row=0, column=0, padx=5, pady=5)

# load image to be "edited"
image = PhotoImage(file="rain.gif")
original_image = image.subsample(3,3) # resize image using subsample
Label(left_frame, image=original_image).grid(row=1, column=0, padx=5, pady=5)

# Display image in right_frame
Label(right_frame, image=image).grid(row=0,column=0, padx=5, pady=5)

# Create tool bar frame
tool_bar = Frame(left_frame, width=180, height=185)
tool_bar.grid(row=2, column=0, padx=5, pady=5)

# Example labels that serve as placeholders for other widgets 
Label(tool_bar, text="Tools", relief=RAISED).grid(row=0, column=0, padx=5, pady=3, ipadx=10) # ipadx is padding inside the Label widget
Label(tool_bar, text="Filters", relief=RAISED).grid(row=0, column=1, padx=5, pady=3, ipadx=10)

# Example labels that could be displayed under the "Tool" menu
Label(tool_bar, text="Select").grid(row=1, column=0, padx=5, pady=5)
Label(tool_bar, text="Crop").grid(row=2, column=0, padx=5, pady=5)
Label(tool_bar, text="Rotate & Flip").grid(row=3, column=0, padx=5, pady=5)
Label(tool_bar, text="Resize").grid(row=4, column=0, padx=5, pady=5)
Label(tool_bar, text="Exposure").grid(row=5, column=0, padx=5, pady=5)

root.mainloop()